## Load Packages

In [46]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 

In [47]:
nlp1 = spacy.load('en_core_web_sm')

## Working of NER

In [48]:
docx1 = nlp1(u"Who is Nishanth?")

In [49]:
for token in docx1.ents:
    print(token.text,token.start_char, token.end_char,token.label_)

In [50]:
docx2 = nlp1(u"Who is Kamal Khumar?")

In [51]:
for token in docx2.ents:
    print(token.text,token.start_char, token.end_char,token.label_)

Kamal Khumar 7 19 PERSON


## Train Data

In [52]:
TRAIN_DATA = [
    ('I am taking COSC 301', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking PHIL 331', {
        'entities': [(12, 20, 'COR')]
    }),
    ('I am taking DATA 301', {
        'entities': [(12, 20, 'COR')]
    })
]

TEST_DATA = [
     ('How is APSC 178?', {
        'entities': [(7, 15, 'PER')]
    }),
     ('Who is COSC 554?', {
        'entities': [(7, 19, 'PER')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    }),
    ('I am taking PHIL 331', {
        'entities': [(12, 20, 'COR')]
    })
]

## Define our variables

In [53]:
model = None
# output_dir=Path("C:\\Users\\nithi\\Documents\\ner")
output_dir=Path("NerModels")
n_iter=100

## Load the model

In [54]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


## Set up the pipeline

In [55]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

## Train the Recognizer

In [56]:
from spacy.training import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 43.79it/s]


{'ner': 4.166666865348816}


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 68.77it/s]


{'ner': 3.9944300055503845}


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 66.37it/s]


...

{'ner': 1.375946801540422e-09}


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 49.22it/s]


{'ner': 1.8166052551224249e-10}


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 48.86it/s]

{'ner': 1.4412927726642419e-10}


## Test the trained model

In [57]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('COSC 301', 'COR')]
Tokens [('I', '', 2), ('am', '', 2), ('taking', '', 2), ('COSC', 'COR', 3), ('301', 'COR', 1)]


## Save the model

In [58]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)        

Saved model to NerModels


## Test the saved model

In [59]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TEST_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from NerModels
Entities [('Nishanth?', 'COR')]
Tokens [('Who', '', 2), ('is', '', 2), ('Nishanth', 'COR', 3), ('?', 'COR', 1)]
Entities [('is Kamal', 'COR')]
Tokens [('Who', '', 2), ('is', 'COR', 3), ('Kamal', 'COR', 1), ('Khumar', '', 2), ('?', '', 2)]
Entities [('.', 'COR')]
Tokens [('I', '', 2), ('like', '', 2), ('London', '', 2), ('and', '', 2), ('Berlin', '', 2), ('.', 'COR', 3)]
Entities [('PHIL 331', 'COR')]
Tokens [('I', '', 2), ('am', '', 2), ('taking', '', 2), ('PHIL', 'COR', 3), ('331', 'COR', 1)]
